In [1]:
import openai
import json
import requests
from openai import OpenAI

import pandas as pd
import glob,os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 142

In [50]:
with open("../assets/openai_api_key.json", 'r') as f:
    api_key = json.load(f)
api_key = api_key['OPENAI_API_KEY']

In [3]:
client = OpenAI(api_key=api_key)

In [121]:
class GPT():
    __classname__ = "OpenAI"
    api_key = ''
    client = None    
    def __init__(self, api_filepath):
        with open(api_filepath,'r') as f:
            ak = json.load(f)
        self.api_key = ak['OPENAI_API_KEY']
        self.client = OpenAI(api_key=self.api_key)
        
    def get_chat_completion(self, msg, model='gpt-4o-mini', **kargs):
        response = self.client.chat.completions.create(
            model = model,
            messages = msg
        )
        return response.choices[0].message.content

    def get_embedding(self, sentence, model="text-embedding-3-small"):
       '''
       - pricing : text-embedding-3-small = $0.02/1M tokens
           텍스트가 많은 pdf는 대략 5,000 tokens -> pdf 200개에 0.02 달러(25~30원).
       text : 한 문장
       return : 한 문장에 대한 embedding (output dimension = 1536)
       '''
       return self.client.embeddings.create(input = sentence, model=model).data[0].embedding

In [124]:
gpt = GPT(api_filepath="../assets/openai_api_key.json")
print(str(gpt.__classname__))

OpenAI


In [4]:
import re
def text_by_pages(filepath):
    ### 문서를 페이지별로 구분하여 list에 저장
    with open(filepath,'r',encoding='utf-8') as f:
        data = f.readlines()
    text = []
    page_text = []
    page_pattern = re.compile('<p.\d*>')
    for d in data:
        # d = d.replace('\n', ' <lf> ') # lf = line feed
        d = d.replace('\n', ' \n') 
        if page_pattern.match(d):
            if len(page_text)>0:
                text.append(' '.join(page_text))
            page_text = [] # 페이지 텍스트 초기화
        page_text.append(d)
    text.append(''.join(page_text)) # 마지막에는 수동으로 추가
    return text

def get_chat_completion(msg, model='gpt-4o-mini',**kargs):
    response = client.chat.completions.create(
        model = model,
        messages = msg
    )
    return response.choices[0].message.content

def erase_page_tag(plist):
    page_pattern = re.compile('<p.\d*>')
    plist = [page_pattern.sub('',x) for x in plist]
    return plist

def extract_text_between_tag(text, tag):
    # Create a regex pattern for the specified tag
    pattern = f'<{tag}>(.*?)</{tag}>'
    # Use re.findall to extract all occurrences between the specified tags
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

In [5]:
sample1 = text_by_pages('./sample1.txt')
print(len(sample1))

22


In [6]:
messages=[
    {"role": "system", 
     "content": """triple quotes 사이에 있는 문서를 참조하여 아래 3가지 작업을 수행하여라. 문서는 pdf 파일을 텍스트로 변환한 것이다. 문서는 <lf> 태그로 구분된 passage들로 구성되어 있다. passage들은 pdf에서 서로 다른 textbox이다. 단, 반복적으로 등장하는 passage는 무의미한 textbox이므로 답변에 포함시키지 말아라.
- 작업1: 문서의 "주제"에 해당하는 paasage를 선택하여라.
- 작업2: 사람의 이름을 추출하여 쉼표(,)로 연결하여라.
- 작업3: 문서의 목차(index)를 추출하여 쉼표(,)로 연결하여라.

3가지 작업 내용에 대해 다음과 같은 형식에 맞춰 답변하여라. 만약 작업의 결과가 명확하지 않다면 해당 결과에는 None을 출력하여라.
<subject>작업1의 결과</subject>
<team>작업2의 결과</team>
<index>작업3의 결과</index>
     """
    },
    {"role": "user", 
     "content": f'"""{" ".join(sample1[:5]) + " ".join(sample1[-3:])}"""'
    }
]
answer1 = get_chat_completion(messages)
print(answer1)

<subject>웹툰 추천 시스템</subject>
<team>강나영, 김나현, 엄성원, 이철민</team>
<index>프로젝트 소개, 데이터 수집 및 전처리, 모델 선택 및 학습, 웹툰 추천 시스템 구현, 결론 및 향후 과제</index>


In [7]:
# 별루...
# messages=[
#     {"role": "system", 
#      "content": """
#      You will be provided with a pair of passages delimited with <lf> tags. Passages represent the content of the project. Answer each given questions in a specified formats.
     
#      Question 1 - What is the main subject of project? Answer in <subject> tag.
#      Question 2 - List the names of the people who participated in the project in <team> tag.
#      Question 3 - What is the table of contents of this project? Answer in <index> tag.

#      Answer in Korean.
#      """
#     },
#     {"role": "user", 
#      "content": f'"""{" ".join(erase_page_tag(sample1[:5]))}"""'
#     }
# ]
# answer = get_chat_completion(messages)
# print(answer)

In [7]:
sample2 = text_by_pages('./sample2.txt')
print(len(sample2))

41


In [8]:
messages=[
    {"role": "system", 
     "content": """triple quotes 사이에 있는 문서를 참조하여 아래 3가지 작업을 수행하여라. 문서는 pdf 파일을 텍스트로 변환한 것이다. 문서는 <lf> 태그로 구분된 passage들로 구성되어 있다. passage들은 pdf에서 서로 다른 textbox이다.ㅍ단, 반복적으로 등장하는 passage는 무의미한 textbox이므로 답변에 포함시키지 말아라.
- 작업1: 문서의 "주제"에 해당하는 paasage를 선택하여라.
- 작업2: 사람의 이름을 추출하여 쉼표(,)로 연결하여라.
- 작업3: 문서의 목차(index)를 추출하여 쉼표(,)로 연결하여라.

3가지 작업 내용에 대해 다음과 같은 형식에 맞춰 답변하여라. 만약 작업의 결과가 명확하지 않다면 해당 결과에는 None을 출력하여라.
<subject>작업1의 결과</subject>
<team>작업2의 결과</team>
<index>작업3의 결과</index>
     """
    },
    {"role": "user", 
     "content": f'"""{" ".join(sample2[:5]) + " ".join(sample2[-3:])}"""'
    }
]
answer2 = get_chat_completion(messages)
print(answer2)

<subject>Netflix Stock Price Prediction with News Topic & Sentiment</subject>  
<team>송규헌, 권도영, 이태경, 김서윤, 한진솔</team>  
<index>INTRODUCTION, DATA PREPROCESSING, MODELING, CONCLUSIONS AND LIMITATION</index>


In [61]:
sample3 = text_by_pages('./sample3.txt')
print(len(sample3))

31


In [62]:
messages = [
	{
	"role" : "system",
	"content" : """triple quotes 사이에 있는 문서를 참조하여 아래 3가지 작업을 수행하여라. 문서는 pdf 파일을 텍스트로 변환한 것이다. <p.숫자>는 문서의 페이지 번호를 의미한다. 문서는 <lf> 태그로 구분된 passage들로 구성되어 있다. passage들은 pdf에서 서로 다른 textbox이다. 단, 반복적으로 등장하는 passage는 무의미한 textbox이므로 답변에 포함시키지 말아라.
- 작업1: 문서의 "주제"에 해당하는 paasage를 선택하여라.
- 작업2: 사람의 이름을 추출하여 쉼표(,)로 연결하여라.
- 작업3: 문서의 목차(table of contents)를 추출하여 쉼표(,)로 연결하여라.

3가지 작업 내용에 대해 다음과 같은 형식에 맞춰 답변하여라. 만약 작업의 결과가 명확하지 않다면 해당 결과에는 None을 출력하여라.
<subject>작업1의 결과</subject>
<team>작업2의 결과</team>
<index>작업3의 결과</index>
     """
   },
	{
	"role" : "user",
	"content" : f'"""{" ".join(sample3[:5]) + " ".join(sample3[-3:])}"""'
	}
]
answer3 = get_chat_completion(messages)
print(answer3)

<subject>비타민 11기 겨울 컨퍼런스 LLM 기반 거짓말 탐지기 : 피의자 신문 언어적 접근</subject>
<team>조민호, 박소연, 박준형, 박세준</team>
<index>서비스 배경 및 기획, 모델 구축 과정, 결론 및 제언</index>


## Clustering
1. Embedding Model, Clustering Algorithm 별로 군집화를 진행
2. SS, CHI 지표를 측정하여 가장 pdf별로 가장 좋은 embedding model, clustering algo 조합 결과를 채택
3. 직접 매긴 target cluster와 비교 -> ARI, HS 지표

In [51]:
from sklearn.decomposition import PCA
def pca_best_component(X):
    '''
    최적의 PCA component값을 찾는 함수
    '''
    pca_optimize = PCA()
    pca_optimize.fit(X)
    
    # 누적 설명된 분산 비율 계산
    cumulative_variance = np.cumsum(pca_optimize.explained_variance_ratio_)
    
    # 99% 이상 설명력을 갖는 주성분 개수 계산
    n_components = np.argmax(cumulative_variance >= 0.99) + 1
    
    # 설명된 분산 비율 시각화
    # plt.figure(figsize=(10, 6))
    # plt.plot(cumulative_variance, marker='o', linestyle='--', color='b')
    # plt.xlabel('Number of Components')
    # plt.ylabel('Cumulative Explained Variance')
    # plt.title('Cumulative Explained Variance by Number of Components')
    # plt.axvline(x=n_components, color='r', linestyle=':', label=f'{n_components} components (95% explained variance)')
    # plt.legend()
    # plt.grid()
    # plt.show()
    # print(f"99% 이상의 설명력을 갖기 위해 {n_components}개의 주성분으로 나누어야 합니다.")

    return n_components

In [52]:
# !pip install kneed
from sklearn.cluster import KMeans
from kneed import KneeLocator

# 클러스터 개수 변화에 따른 SSE 확인, Elbow Point 찾기
def SSE_graph(K, data):
    SSE = []
    k = 1
    while 1 <= k <= K:
        k_means = KMeans(n_clusters = k)     # 클러스터 개수가 k개인 모델 생성
        k_means.fit(data)
        SSE.append(k_means.inertia_)     # inertia : sum of squared distances of samples to their closest cluster center
        k += 1
     
    plt.plot(range(1, K+1), SSE, 'o')     # K값에 따른 SSE 표시
    plt.plot(range(1, K+1), SSE, '--')     # SSE값을 연결하는 직선 그리기
    plt.xlabel('K')
    plt.ylabel('SSE')
    kn = KneeLocator(range(1, K+1), SSE, curve = 'convex', direction = 'decreasing')    
    return SSE

In [53]:
### 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 실루엣 계수를 면적으로 
### 시각화한 함수 작성
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math

def visualize_silhouette(max_cluster, X_features):     
    '''
    K-means, Spectral clustering에서 최적의 K값을 찾는 함수
    '''
    # cluster의 최솟값은 3으로 고정하고 clusterlist를 만든다.
    min_cluster = 4
    cluster_lists = [x for x in range(min_cluster,max_cluster)]
    
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    # fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    results = []
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        # y_lower = 10
        # axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
        #                   'Silhouette Score :' + str(round(sil_avg,3)) )
        # axs[ind].set_xlabel("The silhouette coefficient values")
        # axs[ind].set_ylabel("Cluster label")
        # axs[ind].set_xlim([-0.1, 1])
        # axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        # axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        # axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        # for i in range(n_cluster):
        #     ith_cluster_sil_values = sil_values[cluster_labels==i]
        #     ith_cluster_sil_values.sort()
            
        #     size_cluster_i = ith_cluster_sil_values.shape[0]
        #     y_upper = y_lower + size_cluster_i
            
        #     color = cm.nipy_spectral(float(i) / n_cluster)
        #     axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
        #                         facecolor=color, edgecolor=color, alpha=0.7)
        #     axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        #     y_lower = y_upper + 10
            
            
        # axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

        results.append([sil_avg, np.var([x if x>=0 else 0 for x in sil_values])])

    ### 결과를 토대로 최적의 K 값을 도출
    # 1. 실루엣 계수의 평균과 분산에 대해 MinMax 정규화
    results_df = pd.DataFrame(data=results,columns=['avg','var']) 
    mms = MinMaxScaler()
    results_df[['avg_norm', 'var_norm']] = mms.fit_transform(results_df[['avg', 'var']])
    # 2. 표준화된 var을 1에서 빼서 "분산은 작은 값이 좋음"을 반영
    results_df['var_norm'] = 1 - results_df['var_norm']
    # 3. 가중 평균 계산 (평균에 대한 가중치: 0.65, 분산에 대한 가중치: 0.35 -> 가중치에 대한 근거는 경험적 판단.)
    results_df['score'] = results_df['avg_norm'] * 0.65 + results_df['var_norm'] * 0.35    
    sorted_results = results_df.sort_values('score',ascending=False)
    optimal_k = sorted_results.index[0]+min_cluster
    return optimal_k

### Embedding
- BERT 기반의 LaBSE 모델을 사용하여 문장 임베딩 (max sequence length = 256)
- ConvBERT 모델 : text feature extraction

In [54]:
# !pip install sentence-transformers
# !pip install tf-keras

In [118]:
# 1. Sentence Transformer
from sentence_transformers import SentenceTransformer
se_model = SentenceTransformer('sentence-transformers/LaBSE') # BERT 기반 문장 임베딩 모델
se_model.__classname__ = "LaBSE"

# 2. ConvBERT 
from transformers import AutoTokenizer, AutoModel
convbert_tokenizer = AutoTokenizer.from_pretrained("YituTech/conv-bert-base")
convbert_model = AutoModel.from_pretrained("YituTech/conv-bert-base")
convbert_model.__classname__ = "ConvBERT"

# 3. Canine
from transformers import CanineTokenizer, CanineModel
canine_tokenizer = CanineTokenizer.from_pretrained('google/canine-c')
canine_model = CanineModel.from_pretrained('google/canine-c')
canine_model.__classname__ = "Canine-C"

# 4. OpenAI text-embedding-ada-002
# !pip install -U tiktoken
# import tiktoken
# openai_model = tiktoken.encoding_for_model("gpt-4o")
# from transformers import GPT2Tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('Xenova/text-embedding-ada-002')

# 5. GIST-Embedding v0
gist_model = SentenceTransformer("avsolatorio/GIST-Embedding-v0", 
                            revision=None)
gist_model.__classname__ = "GIST-Embedding-v0"

In [91]:
def get_embedding(text, model="text-embedding-3-small"):
   '''
   - pricing : text-embedding-3-small = $0.02/1M tokens
       텍스트가 많은 pdf는 대략 5,000 tokens -> pdf 200개에 0.02 달러.
   text : 한 문장
   return : 한 문장에 대한 embedding (output dimension = 1536)
   '''
   return client.embeddings.create(input = text, model=model).data[0].embedding
tmpt = [s.replace('\n',' ') for s in erase_page_tag(sample2)]
ees = []
for tt in tmpt:
    tmp = get_embedding(tt)
    ees.append(tmp)

In [92]:
np.array(ees).shape

(41, 1536)

In [125]:
model_dict = {
    'sentence':[se_model, gist_model, gpt],
    'token':[(convbert_tokenizer,convbert_model),(canine_tokenizer,canine_model)]
}

In [126]:
gpt.__classname__

'OpenAI'

In [131]:
from sklearn.cluster import SpectralClustering,KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import torch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import gc

def clustering(text, n_clusters, model_dict, random_state):
    '''
    text : pdf 문서를 page별로 split하고, <p>태그와 \n 문자열을 없앤 텍스트 리스트 전달.
    n_clusters : 클러스터의 개수를 지정 -> extract_text_between_tag(text,'index')로 추출한 인덱스의 개수 + 2

    return
        - total_metrics : 모델별, 클러스터링 알고리즘별 클러스터링 결과를 SS, CHI로 평가지표로 측정한 결과에 대한 데이터프레임
        - total_dict : model_name을 key값으로 text, cluster label, embedding 컬럼으로 하는 데이터프레임을 value로 갖는 딕셔너리
    '''
    total_metrics = pd.DataFrame(columns=['model','kmeans_ss','spectral_ss','kmeans_chi','spectral_chi'])
    total_dict = {}
    max_cluster = len(text)-1
    # sentence 단위로 임베딩하는 모델의 경우
    for model in model_dict['sentence']:
        model_name = model.__classname__
        if model_name == 'Encoding':
            text_embeddings = [model.encode(t) for t in text]
        elif model_name in set(['LaBSE','GIST-Embedding-v0']):
            text_embeddings = model.encode(text, convert_to_tensor=True)
            text_embeddings = text_embeddings.cpu().detach()
        elif model_name == 'OpenAI':
            text_embeddings = []
            for t in text:
                embedding = model.get_embedding(t)
                text_embeddings.append(embedding)
            text_embeddings = np.array(text_embeddings)
        else:
            text_embeddings = model.encode(text)
        ### 차원 축소 : PCA + t-SNE
        # PCA n_component 구하기
        optimal_component = pca_best_component(text_embeddings)
        # PCA
        text_embeddings = PCA(n_components=optimal_component, random_state=random_state).fit_transform(text_embeddings)
        # t-SNE (3차원으로 축소하는 것으로 고정)
        # perplexity는 일반적으로 데이터 개수의 3분의 1 이하
        general_perplexity = int(text_embeddings.shape[0]/3)
        # 최솟값은 5, 최댓값은 50으로 제한
        if general_perplexity<5:
            general_perplexity = 5
        elif general_perplexity>50:
            general_perplexity=50
        text_embeddings = TSNE(n_components=3, perplexity=general_perplexity,
                               random_state=random_state).fit_transform(text_embeddings)
        # 최적의 k값 찾기 [x] >> pdf의 인덱스 개수에 맞춰 n_clusters를 설정
        # optimal_k = visualize_silhouette(max_cluster=max_cluster,X_features=text_embeddings)
        # kmeans와 spectral clustering 진행
        kmeans = KMeans(n_clusters=n_clusters, 
                init='k-means++', # centroid들을 서로 최대한 멀리 배치하는 initialisation 방식
                max_iter = 500,
                random_state = random_state)
        spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors',
                             random_state=random_state)
        kmeans.fit(text_embeddings)
        spectral.fit(text_embeddings)
        # 평가지표 계산
        new_row = pd.DataFrame(data=[[
            model_name, # model name
            silhouette_score(text_embeddings, kmeans.labels_), # kmeans ss
            silhouette_score(text_embeddings, spectral.labels_), # spectral ss
            calinski_harabasz_score(text_embeddings, kmeans.labels_), # kmeans chi
            calinski_harabasz_score(text_embeddings, spectral.labels_) # spectral chi
        ]], columns = total_metrics.columns)

        # Save results
        total_metrics = pd.concat([total_metrics, new_row],axis=0,ignore_index=True)
        temp_df = pd.DataFrame(data=[
            text, kmeans.labels_, spectral.labels_
        ]).transpose()
        temp_df.columns = ['text','kmeans','spectral']
        total_dict[model_name] = pd.concat([
            temp_df,pd.DataFrame(text_embeddings)
        ],axis=1)

        # garbage collection > 별 의미 없는 듯
        # gc.collect()

    # token 단위로 임베딩하는 모델의 경우
    for tokenizer, model in model_dict['token']:
        model_name = model.__classname__
        if model_name == 'Canine-C':
            token = tokenizer(erase_page_tag(text), padding='longest', truncation=True, return_tensors='pt')
        else: # ConvBERT
            token = tokenizer(erase_page_tag(text), padding='longest',return_tensors='pt')
        text_embeddings = model(**token).last_hidden_state.detach()
        flattened = text_embeddings.view(text_embeddings.shape[0],-1)
        ### 차원 축소 : PCA + t-SNE
        # PCA n_component 구하기
        optimal_component = pca_best_component(flattened)
        # PCA
        text_embeddings = PCA(n_components=optimal_component, random_state=random_state).fit_transform(flattened)
        # t-SNE (3차원으로 축소하는 것으로 고정)
        # perplexity는 일반적으로 데이터 개수의 3분의 1 이하
        general_perplexity = int(text_embeddings.shape[0]/3)
        # 최솟값은 5, 최댓값은 50으로 제한
        if general_perplexity<5:
            general_perplexity = 5
        elif general_perplexity>50:
            general_perplexity=50
        text_embeddings = TSNE(n_components=3, perplexity=general_perplexity,
                               random_state=random_state).fit_transform(text_embeddings)

        # 최적의 k값 찾기 [x] >> pdf의 인덱스 개수에 맞춰 n_clusters를 설정
        # optimal_k = visualize_silhouette(max_cluster=max_cluster,X_features=text_embeddings)
        # kmeans와 spectral clustering 진행
        kmeans = KMeans(n_clusters=n_clusters, 
                init='k-means++', # centroid들을 서로 최대한 멀리 배치하는 initialisation 방식
                max_iter = 500,
                random_state = random_state)
        spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors',
                             random_state=random_state)
        kmeans.fit(text_embeddings)
        spectral.fit(text_embeddings)
        # 평가지표 계산
        new_row = pd.DataFrame(data=[[
            model_name, # model name
            silhouette_score(text_embeddings, kmeans.labels_), # kmeans ss
            silhouette_score(text_embeddings, spectral.labels_), # spectral ss
            calinski_harabasz_score(text_embeddings, kmeans.labels_), # kmeans chi
            calinski_harabasz_score(text_embeddings, spectral.labels_) # spectral chi
        ]], columns = total_metrics.columns)

        # Save results
        total_metrics = pd.concat([total_metrics, new_row],axis=0,ignore_index=True)
        temp_df = pd.DataFrame(data=[
            text, kmeans.labels_, spectral.labels_
        ]).transpose()
        temp_df.columns = ['text','kmeans','spectral']
        total_dict[model_name] = pd.concat([
            temp_df,pd.DataFrame(text_embeddings)
        ],axis=1)

        # garbage collection
        # gc.collect()

    return total_metrics, total_dict

In [128]:
def best_model_n_algo(metrics_df):
    """
    metrics_df : clustering함수 결과로 얻은 total_metrics

    return : total_metrics에서 가장 성능이 좋은 model과 클러스터링 알고리즘을 반환 
    -> total_dict에서 해당 모델 이름과 클러스터링 알고리즘 이름으로 클러스터 결과를 찾을 수 있음.
    """
    metrics_df['kmeans']=metrics_df['kmeans_ss']+metrics_df['kmeans_chi']
    metrics_df['spectral']=metrics_df['spectral_ss']+metrics_df['spectral_chi']
    
    max_value = metrics_df[['spectral','kmeans']].max().max()
    # 최대값을 가진 행과 열 찾기
    max_value_row_col = metrics_df[['spectral', 'kmeans']].apply(lambda x: x == max_value).stack()
    max_value_row_col = max_value_row_col[max_value_row_col].index[0]
    model_name = metrics_df.loc[max_value_row_col[0],'model']
    clustering_algo = max_value_row_col[1]
    return model_name, clustering_algo

In [129]:
def renew_cluster_byorder(dfs, best):
    '''
    dfs : clustering 함수 결과로 얻은 total_dict
    best = (best_model, best_algo) 튜플
        - best_model : best_model_n_algo 결과로 얻은 model_name
        - best_algo : best_model_n_algo 결과로 얻은 clustering_algo

    return df : 순서가 엉켜있는 cluster label을 재정렬한 결과를 반환
    '''
    best_model, best_algo = best
    df = dfs[best_model][['text',best_algo]]
    cnum = len(df[best_algo].unique())
    checklist = []
    for c in df[best_algo]:
        if c not in checklist:
            checklist.append(c)
        if len(checklist) == cnum:
            break
    renew_c = list(range(cnum))
    df['new_cluster'] = df[best_algo].apply(lambda x:renew_c[checklist.index(x)])
    return df

### sample별 clustering 결과

In [132]:
import time
start_time = time.time()

### 첫 5페이지에서 요약한 <index> 정보를 통해 n_clusters 설정
s1_nclusters = len(extract_text_between_tag(answer1,'index')[0].split(',')) + 2
s2_nclusters = len(extract_text_between_tag(answer2,'index')[0].split(',')) + 2
s3_nclusters = len(extract_text_between_tag(answer3,'index')[0].split(',')) + 2

# 각 sample text의 page tag와 \n를 없애준다.
sample1_text = [s.replace('\n',' ') for s in erase_page_tag(sample1)]
sample2_text = [s.replace('\n',' ') for s in erase_page_tag(sample2)]
sample3_text = [s.replace('\n',' ') for s in erase_page_tag(sample3)]

s1_metrics, s1_dfs = clustering(text=sample1_text,
                       n_clusters=s1_nclusters,
                       model_dict=model_dict,
                       random_state = RANDOM_STATE)
s2_metrics, s2_dfs = clustering(text=sample2_text,
                       n_clusters=s2_nclusters, 
                       model_dict=model_dict,
                       random_state = RANDOM_STATE)
s3_metrics, s3_dfs = clustering(text=sample3_text,
                       n_clusters=s3_nclusters, 
                       model_dict=model_dict,
                       random_state = RANDOM_STATE)
print(f"{time.time()-start_time:0.2f} seconds")

214.34 seconds


In [133]:
s1_metrics

,model,kmeans_ss,spectral_ss,kmeans_chi,spectral_chi
0,LaBSE,0.143071,0.129194,6.920010,7.010319
1,GIST-Embedding-v0,0.177095,0.059558,8.247603,4.382099
2,OpenAI,0.152055,0.100633,6.379099,5.368058
3,ConvBERT,0.213544,0.042948,9.257685,4.993456
4,Canine-C,0.179941,0.182227,8.512961,6.178575


In [134]:
s2_metrics

,model,kmeans_ss,spectral_ss,kmeans_chi,spectral_chi
0,LaBSE,0.225770,0.150443,24.336221,9.058081
1,GIST-Embedding-v0,0.218695,0.252737,16.262513,16.953961
2,OpenAI,0.231234,0.198090,17.189459,11.637532
3,ConvBERT,0.210228,0.193859,16.267961,15.038704
4,Canine-C,0.204173,0.188609,15.769724,11.682071


In [135]:
s3_metrics

,model,kmeans_ss,spectral_ss,kmeans_chi,spectral_chi
0,LaBSE,0.180692,0.227414,9.786701,11.935285
1,GIST-Embedding-v0,0.209102,0.183199,10.779423,9.693611
2,OpenAI,0.242993,0.254565,12.287375,12.959210
3,ConvBERT,0.304815,0.284957,16.971637,17.315836
4,Canine-C,0.207217,0.260713,11.004939,13.529499


In [136]:
print(f"sample1 :",best_model_n_algo(s1_metrics))
print(f"sample2 :",best_model_n_algo(s2_metrics))
print(f"sample3 :",best_model_n_algo(s3_metrics))

sample1 : ('ConvBERT', 'kmeans')
sample2 : ('LaBSE', 'kmeans')
sample3 : ('ConvBERT', 'spectral')


In [137]:
print(s1_nclusters,s2_nclusters,s3_nclusters)
s1_renew = renew_cluster_byorder(dfs=s1_dfs, best=best_model_n_algo(metrics_df=s1_metrics))
s2_renew = renew_cluster_byorder(dfs=s2_dfs, best=best_model_n_algo(metrics_df=s2_metrics))
s3_renew = renew_cluster_byorder(dfs=s3_dfs, best=best_model_n_algo(metrics_df=s3_metrics))

7 6 5


In [138]:
# 내가 직접 매긴 정답 clustering
s1_renew['target'] = [0,0,1,2,2,2,3,3,3,3,3,3,3,3,3,3,3,4,4,5,5,6]
s2_renew['target'] = [0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5]
s3_renew['target'] = [0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,4]

In [139]:
# ARI, HS
from sklearn.metrics import adjusted_rand_score, homogeneity_score
s1_air = adjusted_rand_score(s1_renew['target'], s1_renew['new_cluster'])
s1_hs = homogeneity_score(s1_renew['target'], s1_renew['new_cluster'])
print(s1_air, s1_hs)

s2_air = adjusted_rand_score(s2_renew['target'], s2_renew['new_cluster'])
s2_hs = homogeneity_score(s2_renew['target'], s2_renew['new_cluster'])
print(s2_air, s2_hs)

s3_air = adjusted_rand_score(s3_renew['target'], s3_renew['new_cluster'])
s3_hs = homogeneity_score(s3_renew['target'], s3_renew['new_cluster'])
print(s3_air, s3_hs)

0.1966148852306979 0.6195675237327858
0.4258502217969905 0.5349039903379956
0.002308940610126665 0.22930542090012493


In [146]:
s1_renew

,text,kmeans,new_cluster,target
0,BITAmin 24-1R 학기 프로젝트 독자와 웹툰 간 로그데이터를 활...,5,0,0
1,BITAmin TABLE OF CONTENTS 목차 소개 01 프로...,1,1,0
2,01 프로젝트 소개 프로젝트 배경 정제된 '대량의 데이터'를 사용하여...,0,2,1
3,02 데이터 수집 및 전처리 BITAmin 데이터 소스 설명,5,0,2
4,02 데이터 수집 및 전처리 BITAmin 데이터 전처리,2,3,2
5,02 데이터 수집 및 전처리 BITAmin 데이터 전처리 피봇 테이...,5,0,2
6,03 모델 선택 및 학습 - CF BITAmin CF(Collabora...,3,4,3
7,03 모델 선택 및 학습 - CF BITAmin CF(Collabora...,6,5,3
8,03 모델 선택 및 학습 - NCF BITAmin NCF(Neural ...,6,5,3
9,03 모델 선택 및 학습 - NCF BITAmin NCF(Neural ...,6,5,3


In [45]:
s2_renew

,text,kmeans,new_cluster,target
0,<p.1> \n 2024 BITAmin 겨울 연합프로젝트 시계열 1조 \n Netf...,4,0,0
1,<p.2> \n CONTENTS \n 01. INTRODUCTION \n 02. D...,1,1,0
2,<p.3> \n 01. INTRODUCTION \n \n,1,1,1
3,<p.4> \n 01. INTRODUCTION \n 1.1 Background of...,5,2,1
4,<p.5> \n 01. INTRODUCTION \n 1.1 Background of...,2,3,1
5,<p.6> \n 01. INTRODUCTION \n 1.2 Brief Project...,5,2,1
6,<p.7> \n 01. INTRODUCTION \n 1.3 Data collecti...,4,0,1
7,<p.8> \n 01. INTRODUCTION \n 1.3 Data collecti...,2,3,1
8,<p.9> \n 01. INTRODUCTION \n 1.3 Data collecti...,4,0,1
9,<p.10> \n 02. DATA PREPROCESSING \n \n,1,1,2


In [141]:
s3_renew

,text,spectral,new_cluster,target
0,비타민 11기 겨울 컨퍼런스 LLM 기반 거짓말 탐지기 : 피의자 신문...,0,0,0
1,서비스 배경 및 기획 | 모델 구축 과정 | 결론 및 제언 서비스 배경 및...,4,1,0
2,서비스 배경 및 기획 ► 문제 상황 비타민 11기 겨울 컨퍼런스 거짓말...,4,1,1
3,서비스 배경 및 기획 ► 문제 상황 비타민 11기 겨울 컨퍼런스 비언어...,2,2,1
4,서비스 배경 및 기획 ► 문제 상황 비타민 11기 겨울 컨퍼런스 단순 ...,0,0,1
5,서비스 배경 및 기획 ► 서비스 제안 및 사용 예시 비타민 11기 겨울 컨...,1,3,1
6,서비스 배경 및 기획 ► 실무 파이프라인 비타민 11기 겨울 컨퍼런스 ...,0,0,1
7,RAG Retrieval –Augmented Generation 사용자의 ...,1,3,1
8,PEFT Parameter Efficient Fine Tuning 적은 매...,1,3,1
9,비타민 11기 겨울 컨퍼런스 서비스 배경 및 기획 | 모델 구축 과정 | ...,4,1,2


In [56]:
s1_dict = dict(zip(list(range(s1_nclusters)),['' for _ in range(s1_nclusters)]))
for idx in s1_renew.index:
    new_cluster = s1_renew.iloc[idx]['new_cluster']
    s1_dict[new_cluster] += s1_renew.iloc[idx]['text']
s1_dict

{0: '<p.1> \n BITAmin \n 24-1R 학기 프로젝트 \n 독자와 웹툰 간 로그데이터를 활용한 \n 웹툰 추천 시스템 \n 2024.06.05 \n RecSys Team 2 | 강나영 김나현 엄성원 이철민 \n  \n<p.11> \n 03 모델 선택 및 학습 - SVD \n BITAmin \n SVD(Singular Value Decomposition) 모델 \n 개념 \n 사용자-아이템 행렬을 세 개의 행렬로 분해하여  \n 사용자와 아이템의 잠재적 특성 (latent factors)을 추출하는 방식. \n 특징  \n m x n 행렬로도 대각행렬을 통한 특이값 분해가 가능하기 때문에  \n 유저, 아이템의 행과 열의 개수가 다른 추천모델에도 적합하여 \n 잠재 요인을 이용하여 사용자가 아직 평가하지 않은 아이템에 대한 평점을 예측 \n  \n 장점: 차원 축소를 통해 계산 효율성 증가 및 노이즈 제거 \n 단점: 큰 데이터셋에서 계산 비용이 높아질 수 있음, 실시간 업데이트 어려움 \n  \n<p.18>  \n 04 웹툰 추천 시스템 구현 - 기존 사용자  \n BITAmin \n 사용자가 이전에 봤던 웹툰: \n 올가미(17회) \n 외모지상주의(16회) \n 이상한 변호사 우영우(7회) \n  \n 타입 스토리 \n 장르 판타지, 드라마 등 \n  \n 웹툰 추천 결과  \n rmse: 0.10001713 \n  \n<p.19>  \n 04 웹툰 추천 시스템 구현 - 신규 사용자  \n BITAmin \n 좋아하는 웹툰 3개 입력: \n 이별 후 사내 결혼 \n 순정말고 순종 \n 다시 쓰는 연애사 \n  \n 타입 스토리 \n 장르 로맨스 등 \n  \n 웹툰 추천 결과  \n rmse: 0.10001718 \n  \n',
 1: '<p.2> \n BITAmin \n TABLE OF CONTENTS \n 목차 소개 \n 01 프로젝트 소개 프로젝트 배경 프로젝트 목표  \n 02 데이터 수집 및 전처리 데이터 소스 설명 데이터 전처리 

In [66]:
extract_text_between_tag(answer1,'index')[0].split(',')

['프로젝트 소개', ' 데이터 수집 및 전처리', ' 모델 선택 및 학습', ' 웹툰 추천 시스템 구현', ' 결론 및 향후 과제']

In [72]:
page_pattern = re.compile(r'<p.\d*>')
last_key = len(s1_dict)-1
summaries1 = []
s1_index_list = extract_text_between_tag(answer1,'index')[0].split(',')
i = 0
for key in s1_dict:
    if key in [0,last_key]:
        continue
    s1_index_item = s1_index_list[i]
    i += 1
    ctext = s1_dict[key]
    ctext = re.sub(r'<p.\d*>', ' ', ctext)
    ctext = ctext.replace('\n', '')

    messages = [
    	{
    	"role" : "system",
    	"content" : f"""Your task is to 'Summarize' the given text in triple quote. Summarize the text shortly focusing on the key points related to '{s1_index_item}'.
         Extract the Subtiles as you can, and summarize the content for each subtitles.
         Answer in Korean! And Answer in following format.

         <main>{s1_index_item}</main
         <subtitle>Extract subtitle</subtitle>
         <content>Summarization result</content>         
         """
       },
    	{
    	"role" : "user",
    	"content" : f'"""{ctext}"""'
    	}
    ]
    
    summary1 = get_chat_completion(messages)
    summaries1.append(summary1)

In [75]:
for s in summaries1:
    print(s)

<main>프로젝트 소개</main>
<subtitle>프로젝트 배경</subtitle>
<content>BITAmin 프로젝트는 사용자 맞춤형 추천 시스템을 구현하기 위해 시작되었으며, 웹툰 추천에 중점을 두고 있습니다.</content>         
<subtitle>프로젝트 목표</subtitle>
<content>이 프로젝트의 목표는 사용자와 아이템 간의 상호작용 데이터를 분석하여 효과적인 추천 알고리즘을 개발하는 것입니다.</content>
<main> 데이터 수집 및 전처리</main>
<subtitle>BITAmin 데이터 소스 설명</subtitle>
<content>프로젝트는 사용자 및 웹툰 데이터를 활용하여 개인화된 추천 시스템을 개발하는 목적을 가지고 있다. 이를 위해 국내 최신 데이터를 사용하며, 데이터 수집은 BITAmin 플랫폼을 통해 이루어진다.</content>
<subtitle>BITAmin 데이터 전처리</subtitle>
<content>수집된 데이터는 적절히 전처리되어야 하며, 이는 추천 알고리즘의 효과성을 높이고 사용자에게 맞춤형 추천을 제공하기 위한 중요한 단계이다. 전처리 과정에서는 데이터의 정제 및 형식 통일성이 중요하다.</content>
<main> 모델 선택 및 학습</main>
<subtitle>NCF(Neural Collaborative Filtering)</subtitle>
<content>BITAmin을 사용한 NCF 모델링을 통해 결과를 출력하였다.</content>
<subtitle>최종 모델 선정</subtitle>
<content>ALS(Alternative Least Squares)를 최종 모델로 선정한 이유는 implicit feedback 데이터의 문제점을 해결하기 위해서이다. 사용자의 선호를 정확히 파악하기 어려운 점을 고려하여 binary 분류 방법과 implicit feedback을 위한 MF의 목적함수를 활용하였다. ALS 모델은 대규모 데이터에 적합하며 RMSE 수치가 0.1로 우

In [145]:
pdf = 'ocr_samples_txt'
ROOT = 'bitamin_auto_readme_generator'

root_absdir = os.getcwd().split(ROOT)[0]+ROOT
# print(root_absdir)
pdf_dir = os.path.join(root_absdir,'data','object_detection','output',pdf)
print(pdf_dir)
for f in os.listdir(pdf_dir):
    print(f)

D:\SKH\Github_Projects\bitamin_auto_readme_generator\data\object_detection\output\ocr_samples_txt
arima_text.txt
asiancup_text.txt
barbot_text.txt
braintumor_text.txt
hangang_text.txt
interior_text.txt
kospi_text.txt
lier-detector_text.txt
netflix_text.txt
webtoon_text.txt
